In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Machine Learning Libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

# Data Augmentation
from imblearn.over_sampling import SMOTE
from scipy import stats
from sklearn.neighbors import NearestNeighbors

# Model Persistence
import joblib
import pickle

# Set random seed untuk reproducibility
np.random.seed(42)

print("=== XGBOOST CYCLING SPEED PREDICTION MODEL ===")
print("Memuat dan memproses data...")

df = pd.DataFrame(data)

print(f"Data awal: {len(df)} baris")
print(f"Columns: {list(df.columns)}")

# =============================================================================
# 2. DATA CLEANING DAN FEATURE ENGINEERING
# =============================================================================

# Handling anomali curah hujan (888.0 mm)
print(f"\nAnomali curah hujan ditemukan: {df['Curah_Hujan'].max()} mm")
df.loc[df['Curah_Hujan'] == 888.0, 'Curah_Hujan'] = df['Curah_Hujan'].median()
print(f"Diganti dengan median: {df['Curah_Hujan'].median()} mm")

# Convert datetime features
df['Tanggal'] = pd.to_datetime(df['Tanggal'], format='%d/%m/%Y')
df['Jam_Mulai_Datetime'] = pd.to_datetime(df['Jam_Mulai'], format='%H:%M').dt.time

# Extract time-based features
df['Hour'] = pd.to_datetime(df['Jam_Mulai'], format='%H:%M').dt.hour
df['Is_Morning'] = (df['Hour'] < 12).astype(int)
df['Day_of_Week'] = df['Tanggal'].dt.dayofweek
df['Is_Weekend'] = (df['Day_of_Week'].isin([5, 6])).astype(int)

# Create categorical features
df['Time_Category'] = df['Hour'].apply(lambda x: 'Early_Morning' if x < 8 
                                       else 'Morning' if x < 12 
                                       else 'Afternoon' if x < 17 
                                       else 'Evening')

# Elevation categories
df['Elevation_Category'] = pd.cut(df['Elevasi'], 
                                  bins=[0, 150, 250, 400, float('inf')],
                                  labels=['Flat', 'Rolling', 'Hilly', 'Mountainous'])

# Rain categories  
df['Rain_Category'] = pd.cut(df['Curah_Hujan'],
                             bins=[-0.1, 0, 10, 30, float('inf')],
                             labels=['No_Rain', 'Light', 'Moderate', 'Heavy'])

# Distance categories
df['Distance_Category'] = pd.cut(df['Jarak'],
                                 bins=[0, 20, 30, 40, float('inf')],
                                 labels=['Short', 'Medium', 'Long', 'Ultra'])

# Sleep quality categories
df['Sleep_Quality'] = pd.cut(df['Jam_Tidur'],
                             bins=[0, 4, 6, 8, float('inf')],
                             labels=['Poor', 'Moderate', 'Good', 'Excellent'])

# Advanced feature engineering
df['Speed_per_Elevation'] = df['Kec_Rata_Rata'] / (df['Elevasi'] + 1)
df['Distance_per_Duration'] = df['Jarak'] / df['Durasi_Menit']
df['Elevation_per_Distance'] = df['Elevasi'] / df['Jarak']
df['Rest_Factor'] = df['Jam_Tidur'] / df['Durasi_Menit'] * 100

# Weather impact factor
df['Weather_Impact'] = df['Curah_Hujan'].apply(lambda x: 1 if x == 0 else 0.8 if x <= 10 else 0.6 if x <= 30 else 0.4)

print(f"\nFeature engineering completed. New features:")
new_features = [col for col in df.columns if col not in data.keys()]
for feature in new_features:
    print(f"  - {feature}")

# =============================================================================
# 3. EXPLORATORY DATA ANALYSIS
# =============================================================================

print("\n=== EXPLORATORY DATA ANALYSIS ===")

# Basic statistics
print("\nStatistik Dasar Target Variable (Kec_Rata_Rata):")
print(f"Mean: {df['Kec_Rata_Rata'].mean():.2f} km/h")
print(f"Std: {df['Kec_Rata_Rata'].std():.2f} km/h")
print(f"Min: {df['Kec_Rata_Rata'].min():.2f} km/h")
print(f"Max: {df['Kec_Rata_Rata'].max():.2f} km/h")

# Correlation analysis
numeric_features = ['Elevasi', 'Jarak', 'Curah_Hujan', 'Jam_Tidur', 'Hour', 'Day_of_Week',
                    'Speed_per_Elevation', 'Distance_per_Duration', 'Elevation_per_Distance', 
                    'Rest_Factor', 'Weather_Impact']

print(f"\nKorelasi dengan target variable:")
correlations = df[numeric_features + ['Kec_Rata_Rata']].corr()['Kec_Rata_Rata'].sort_values(key=abs, ascending=False)
for feature, corr in correlations.items():
    if feature != 'Kec_Rata_Rata':
        print(f"  {feature}: {corr:.3f}")

# =============================================================================
# 4. DATA PREPARATION
# =============================================================================

print("\n=== DATA PREPARATION ===")

# Select features for modeling
categorical_features = ['Time_Category', 'Elevation_Category', 'Rain_Category', 
                       'Distance_Category', 'Sleep_Quality']

# Encode categorical variables
le_encoders = {}
for cat_feature in categorical_features:
    le = LabelEncoder()
    df[f'{cat_feature}_encoded'] = le.fit_transform(df[cat_feature].astype(str))
    le_encoders[cat_feature] = le

# Final feature selection
feature_columns = (numeric_features + 
                  [f'{cat}_encoded' for cat in categorical_features] +
                  ['Is_Morning', 'Is_Weekend'])

X = df[feature_columns].copy()
y = df['Kec_Rata_Rata'].copy()

print(f"Features selected: {len(feature_columns)}")
print(f"Dataset shape: {X.shape}")

# =============================================================================
# 5. ADVANCED DATA AUGMENTATION
# =============================================================================

print("\n=== DATA AUGMENTATION ===")

class AdvancedDataAugmentation:
    def __init__(self, X, y, random_state=42):
        self.X = X
        self.y = y
        self.random_state = random_state
        np.random.seed(random_state)
    
    def gaussian_noise_augmentation(self, noise_factor=0.1, n_samples=50):
        """Add Gaussian noise to create new samples"""
        X_noise = []
        y_noise = []
        
        for _ in range(n_samples):
            idx = np.random.randint(0, len(self.X))
            sample = self.X.iloc[idx].values.copy()
            target = self.y.iloc[idx]
            
            # Add noise to continuous features only
            continuous_features = ['Elevasi', 'Jarak', 'Curah_Hujan', 'Jam_Tidur', 'Hour',
                                 'Speed_per_Elevation', 'Distance_per_Duration', 
                                 'Elevation_per_Distance', 'Rest_Factor', 'Weather_Impact']
            
            for i, feature in enumerate(self.X.columns):
                if feature in continuous_features:
                    noise = np.random.normal(0, noise_factor * np.std(self.X[feature]))
                    sample[i] += noise
                    sample[i] = max(0, sample[i])  # Ensure non-negative values
            
            X_noise.append(sample)
            # Adjust target based on changes
            y_adjustment = np.random.normal(0, 0.5)
            y_noise.append(max(10, target + y_adjustment))  # Min speed 10 km/h
        
        return np.array(X_noise), np.array(y_noise)
    
    def interpolation_augmentation(self, n_samples=30):
        """Create new samples by interpolating between existing ones"""
        X_interp = []
        y_interp = []
        
        for _ in range(n_samples):
            idx1, idx2 = np.random.choice(len(self.X), 2, replace=False)
            alpha = np.random.uniform(0.2, 0.8)
            
            sample1 = self.X.iloc[idx1].values
            sample2 = self.X.iloc[idx2].values
            target1 = self.y.iloc[idx1]
            target2 = self.y.iloc[idx2]
            
            # Interpolate
            new_sample = alpha * sample1 + (1 - alpha) * sample2
            new_target = alpha * target1 + (1 - alpha) * target2
            
            X_interp.append(new_sample)
            y_interp.append(new_target)
        
        return np.array(X_interp), np.array(y_interp)
    
    def seasonal_variation_augmentation(self, n_samples=20):
        """Create variations based on seasonal/weather patterns"""
        X_seasonal = []
        y_seasonal = []
        
        for _ in range(n_samples):
            idx = np.random.randint(0, len(self.X))
            sample = self.X.iloc[idx].values.copy()
            target = self.y.iloc[idx]
            
            # Simulate different weather conditions
            weather_variation = np.random.choice(['sunny', 'windy', 'humid'])
            
            if weather_variation == 'windy':
                # Reduce speed slightly
                target *= np.random.uniform(0.95, 1.0)
            elif weather_variation == 'humid':
                # Reduce speed more
                target *= np.random.uniform(0.9, 0.98)
            else:  # sunny
                # Slight speed increase
                target *= np.random.uniform(1.0, 1.05)
            
            X_seasonal.append(sample)
            y_seasonal.append(target)
        
        return np.array(X_seasonal), np.array(y_seasonal)

# Apply augmentation
augmenter = AdvancedDataAugmentation(X, y)

# Generate augmented data
X_noise, y_noise = augmenter.gaussian_noise_augmentation(n_samples=60)
X_interp, y_interp = augmenter.interpolation_augmentation(n_samples=40)
X_seasonal, y_seasonal = augmenter.seasonal_variation_augmentation(n_samples=30)

# Combine all data
X_augmented = np.vstack([X.values, X_noise, X_interp, X_seasonal])
y_augmented = np.hstack([y.values, y_noise, y_interp, y_seasonal])

print(f"Original data: {len(X)} samples")
print(f"Augmented data: {len(X_augmented)} samples")
print(f"Augmentation ratio: {len(X_augmented)/len(X):.1f}x")

# Convert back to DataFrame for consistency
X_augmented = pd.DataFrame(X_augmented, columns=X.columns)
y_augmented = pd.Series(y_augmented)

# =============================================================================
# 6. MODEL TRAINING DAN OPTIMIZATION
# =============================================================================

print("\n=== MODEL TRAINING & OPTIMIZATION ===")

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_augmented, y_augmented, test_size=0.2, random_state=42, stratify=None
)

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# XGBoost parameter tuning
print("\nOptimizing XGBoost hyperparameters...")

# Initial XGBoost model
xgb_model = xgb.XGBRegressor(random_state=42, n_jobs=-1)

# Define parameter grid for GridSearch
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [0, 0.1, 0.5]
}

# Perform GridSearch with cross-validation
grid_search = GridSearchCV(
    xgb_model, param_grid, cv=5, scoring='neg_mean_absolute_error',
    n_jobs=-1, verbose=1
)

grid_search.fit(X_train_scaled, y_train)

print(f"Best parameters found:")
for param, value in grid_search.best_params_.items():
    print(f"  {param}: {value}")

# Train final model with best parameters
final_model = grid_search.best_estimator_

# =============================================================================
# 7. MODEL EVALUATION
# =============================================================================

print("\n=== MODEL EVALUATION ===")

# Predictions
y_pred_train = final_model.predict(X_train_scaled)
y_pred_test = final_model.predict(X_test_scaled)

# Calculate metrics
train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"Training Performance:")
print(f"  MAE: {train_mae:.3f} km/h")
print(f"  R²: {train_r2:.3f}")
print(f"  RMSE: {train_rmse:.3f} km/h")

print(f"\nTest Performance:")
print(f"  MAE: {test_mae:.3f} km/h")
print(f"  R²: {test_r2:.3f}")
print(f"  RMSE: {test_rmse:.3f} km/h")

# Cross-validation
cv_scores = cross_val_score(final_model, X_train_scaled, y_train, 
                           cv=5, scoring='neg_mean_absolute_error')
print(f"\nCross-validation MAE: {-cv_scores.mean():.3f} ± {cv_scores.std():.3f}")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\nTop 10 Most Important Features:")
for i, (_, row) in enumerate(feature_importance.head(10).iterrows()):
    print(f"  {i+1}. {row['feature']}: {row['importance']:.3f}")

# =============================================================================
# 8. MODEL PERSISTENCE
# =============================================================================

print("\n=== SAVING MODEL ===")

# Create model package with all necessary components
model_package = {
    'model': final_model,
    'scaler': scaler,
    'label_encoders': le_encoders,
    'feature_columns': feature_columns,
    'feature_importance': feature_importance,
    'model_performance': {
        'test_mae': test_mae,
        'test_r2': test_r2,
        'test_rmse': test_rmse,
        'cv_mae_mean': -cv_scores.mean(),
        'cv_mae_std': cv_scores.std()
    },
    'training_info': {
        'original_samples': len(X),
        'augmented_samples': len(X_augmented),
        'best_params': grid_search.best_params_,
        'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
}

# Save model
joblib.dump(model_package, 'cycling_speed_prediction_model.joblib')
print("Model saved as 'cycling_speed_prediction_model.joblib'")

# =============================================================================
# 9. PREDICTION EXAMPLE
# =============================================================================

print("\n=== PREDICTION EXAMPLE ===")

def predict_cycling_speed(elevasi, jarak, curah_hujan, jam_tidur, jam_mulai, 
                         model_package=model_package):
    """
    Prediksi kecepatan rata-rata bersepeda
    
    Parameters:
    - elevasi: elevasi dalam meter
    - jarak: jarak dalam km
    - curah_hujan: curah hujan dalam mm
    - jam_tidur: jam tidur dalam jam
    - jam_mulai: jam mulai (format: 'HH:MM')
    """
    
    # Extract components
    model = model_package['model']
    scaler = model_package['scaler']
    le_encoders = model_package['label_encoders']
    feature_columns = model_package['feature_columns']
    
    # Create input data
    hour = pd.to_datetime(jam_mulai, format='%H:%M').hour
    is_morning = 1 if hour < 12 else 0
    day_of_week = 1  # Default weekday
    is_weekend = 0
    
    # Create categorical features
    time_category = ('Early_Morning' if hour < 8 
                    else 'Morning' if hour < 12 
                    else 'Afternoon' if hour < 17 
                    else 'Evening')
    
    elevation_category = ('Flat' if elevasi <= 150 
                         else 'Rolling' if elevasi <= 250
                         else 'Hilly' if elevasi <= 400
                         else 'Mountainous')
    
    rain_category = ('No_Rain' if curah_hujan <= 0
                    else 'Light' if curah_hujan <= 10
                    else 'Moderate' if curah_hujan <= 30
                    else 'Heavy')
    
    distance_category = ('Short' if jarak <= 20
                        else 'Medium' if jarak <= 30
                        else 'Long' if jarak <= 40
                        else 'Ultra')
    
    sleep_quality = ('Poor' if jam_tidur <= 4
                    else 'Moderate' if jam_tidur <= 6
                    else 'Good' if jam_tidur <= 8
                    else 'Excellent')
    
    # Create feature vector
    durasi_estimasi = jarak / 20 * 60  # Estimate duration
    
    input_data = {
        'Elevasi': elevasi,
        'Jarak': jarak,
        'Curah_Hujan': curah_hujan,
        'Jam_Tidur': jam_tidur,
        'Hour': hour,
        'Day_of_Week': day_of_week,
        'Speed_per_Elevation': 20 / (elevasi + 1),  # Default assumption
        'Distance_per_Duration': jarak / durasi_estimasi,
        'Elevation_per_Distance': elevasi / jarak,
        'Rest_Factor': jam_tidur / durasi_estimasi * 100,
        'Weather_Impact': (1 if curah_hujan == 0 else 
                          0.8 if curah_hujan <= 10 else 
                          0.6 if curah_hujan <= 30 else 0.4),
        'Time_Category_encoded': le_encoders['Time_Category'].transform([time_category])[0],
        'Elevation_Category_encoded': le_encoders['Elevation_Category'].transform([elevation_category])[0],
        'Rain_Category_encoded': le_encoders['Rain_Category'].transform([rain_category])[0],
        'Distance_Category_encoded': le_encoders['Distance_Category'].transform([distance_category])[0],
        'Sleep_Quality_encoded': le_encoders['Sleep_Quality'].transform([sleep_quality])[0],
        'Is_Morning': is_morning,
        'Is_Weekend': is_weekend
    }
    
    # Convert to DataFrame
    input_df = pd.DataFrame([input_data])
    input_df = input_df[feature_columns]
    
    # Scale and predict
    input_scaled = scaler.transform(input_df)
    prediction = model.predict(input_scaled)[0]
    
    return prediction

# Test prediction
print("Testing prediction function:")
test_speed = predict_cycling_speed(
    elevasi=200,
    jarak=25,
    curah_hujan=0,
    jam_tidur=7,
    jam_mulai='06:00'
)
print(f"Predicted speed: {test_speed:.2f} km/h")

print(f"\n{'='*60}")
print("MODEL TRAINING COMPLETED SUCCESSFULLY!")
print(f"{'='*60}")
print(f"Model Performance Summary:")
print(f"  - Test MAE: {test_mae:.3f} km/h")
print(f"  - Test R²: {test_r2:.3f}")
print(f"  - Model saved as: cycling_speed_prediction_model.joblib")
print(f"{'='*60}")

=== XGBOOST CYCLING SPEED PREDICTION MODEL ===
Memuat dan memproses data...
Data awal: 42 baris
Columns: ['Tanggal', 'Jam_Mulai', 'Durasi_Menit', 'Elevasi', 'Jarak', 'Kec_Rata_Rata', 'Kec_Maksimal', 'Curah_Hujan', 'Jam_Tidur']

Anomali curah hujan ditemukan: 888.0 mm
Diganti dengan median: 0.1 mm

Feature engineering completed. New features:
  - Jam_Mulai_Datetime
  - Hour
  - Is_Morning
  - Day_of_Week
  - Is_Weekend
  - Time_Category
  - Elevation_Category
  - Rain_Category
  - Distance_Category
  - Sleep_Quality
  - Speed_per_Elevation
  - Distance_per_Duration
  - Elevation_per_Distance
  - Rest_Factor
  - Weather_Impact

=== EXPLORATORY DATA ANALYSIS ===

Statistik Dasar Target Variable (Kec_Rata_Rata):
Mean: 20.46 km/h
Std: 2.88 km/h
Min: 15.20 km/h
Max: 27.20 km/h

Korelasi dengan target variable:
  Jarak: 0.649
  Elevasi: 0.485
  Hour: -0.455
  Rest_Factor: -0.318
  Day_of_Week: -0.296
  Distance_per_Duration: 0.181
  Jam_Tidur: 0.167
  Speed_per_Elevation: -0.130
  Curah_Hujan